# NGram Language Models

```yaml
Course:  DS 5001
Module:  03 Lab
Topic:   NGram Language Models
Version: 1
Author:  R.C. Alvarado
Date:    12 December 2023
```

## Purpose 

We now create a series of simple n-gram langage models from our small corpus and evaluate them.

## Pattern

1. Import corpus &rarr; `TOKEN`, `VOCAB`.
2. Extract ngrams from training tokens &rarr; `NGRAM`.
3. Count ngrams and convert to models &rarr; `MODEL`.
4. Convert test sentences into tokens &rarr; `TEST_SENT`, `TEST_TOKEN`.
5. Extract ngrams from test tokens &rarr; `TEST_NGRAM`.
6. Test model by joining model information `M.i` to `TEST_NGRAM` and then summing i per sentence &rarr; `TEST_NGRAM'`, `TEST_SENT'`.
7. Compute model perplexity by averaging sentence information sums and exponentiating. 

## Set Up

### Import libraries

In [293]:
import pandas as pd
import numpy as np

### Configure

In [294]:
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_dir = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [295]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']
path_prefix = f"{output_dir}/austen-combo"
n = 3

## Get Data

We grab our corpus of two novels.

In [296]:
VOCAB = pd.read_csv(f"{path_prefix}-VOCAB.csv").set_index('term_str')
TOKEN = pd.read_csv(f"{path_prefix}-TOKENS.csv").set_index(OHCO)

## Generate Models

This function generates models up to the length specified.

Our approach is to bind the sequence of term strings in TOKEN to itself with an offset of 1 for each value of $n$.

In [297]:
TOKEN

token_str     term_str
book_id chap_num para_num sent_num token_num                          
1       1        0        0        0                  Sir          sir
                                   1               Walter       walter
                                   2               Elliot       elliot
                                   3                   of           of
                                   4             Kellynch     kellynch
...                                                   ...          ...
2       50       23       0        8                  and          and
                                   9          Sensibility  sensibility
                                   10                  by           by
                                   11                Jane         jane
                                   12              Austen       austen

[204801 rows x 2 columns]

In [331]:
def get_ngrams(TOKEN, n=2, sent_key='sent_num'):

    OHCO = TOKEN.index.names
    grouper = list(OHCO)[:OHCO.index(sent_key)+1]

    PADDED = TOKEN.groupby(grouper)\
        .apply(lambda x: '<s> ' + ' '.join(x.term_str) + ' <s>')\
        .apply(lambda x: pd.Series(x.split()))\
        .stack().to_frame('term_str')
    PADDED.index.names = grouper + ['token_num']

    for i in range(1, n):
        PADDED = PADDED.join(PADDED.term_str.shift(-i), rsuffix=i)

    PADDED.columns = [f'w{j}' for j in range(n)]

    return PADDED

In [332]:
NG = get_ngrams(TOKEN, n=3)

In [337]:
NG.fillna('<s>')

w0        w1        w2
book_id chap_num para_num sent_num token_num                                 
1       1        0        0        0                  <s>       sir    walter
                                   1                  sir    walter    elliot
                                   2               walter    elliot        of
                                   3               elliot        of  kellynch
                                   4                   of  kellynch      hall
...                                                   ...       ...       ...
2       50       23       0        10         sensibility        by      jane
                                   11                  by      jane    austen
                                   12                jane    austen       <s>
                                   13              austen       <s>       <s>
                                   14                 <s>       <s>       <s>

[232051 rows x 3 columns]

In [338]:
NG.iloc[:,:3]

w0        w1        w2
book_id chap_num para_num sent_num token_num                                 
1       1        0        0        0                  <s>       sir    walter
                                   1                  sir    walter    elliot
                                   2               walter    elliot        of
                                   3               elliot        of  kellynch
                                   4                   of  kellynch      hall
...                                                   ...       ...       ...
2       50       23       0        10         sensibility        by      jane
                                   11                  by      jane    austen
                                   12                jane    austen       <s>
                                   13              austen       <s>      None
                                   14                 <s>      None      None

[232051 rows x 3 columns]

In [339]:
def get_ngram_counts(NGRAM):
    "Compress the sequences into counts"
    
    n = len(NGRAM.columns)
    C = [None for i in range(n)]
    
    for i in range(n):

        # Count distinct ngrams
        C[i] = NGRAM.iloc[:, :i+1].value_counts().to_frame('n').sort_index()
    
        # Get joint probabilities (MLE)
        C[i]['p'] = C[i].n / C[i].n.sum()
        C[i]['i'] = np.log2(1/C[i].p)

        # Get conditional probabilities (MLE)
        if i > 0:
            C[i]['cp'] = C[i].n / C[i-1].n
            C[i]['ci'] = np.log2(1/C[i].cp)
            
    return C

Generate unigram, bigram, and trigram models.

In [340]:
M = get_ngram_counts(NG)

In [341]:
M[0].sort_values('n')

,n,p,i
w0,,,
scolding,1,0.000004,17.824082
seducer,1,0.000004,17.824082
diversified,1,0.000004,17.824082
seduced,1,0.000004,17.824082
diverting,1,0.000004,17.824082
...,...,...,...
of,6146,0.026486,5.238650
and,6290,0.027106,5.205238
to,6923,0.029834,5.066901


In [342]:
M[1].sample(5)

,,n,p,i,cp,ci
w0,w1,,,,,
any,creature,1,0.000004,17.824076,0.001695,9.204571
am,sorry,13,0.000056,14.123636,0.031707,4.979040
and,laugh,2,0.000009,16.824076,0.000318,11.618844
exquisite,grace,1,0.000004,17.824076,0.100000,3.321928
the,importunate,1,0.000004,17.824076,0.000134,12.860117


In [343]:
M[2].sample(5)

,,,n,p,i,cp,ci
w0,w1,w2,,,,,
little,way,our,1,0.000004,17.82407,0.500000,1.000000
<s>,was,she,1,0.000004,17.82407,0.032258,4.954196
not,enough,to,1,0.000004,17.82407,0.500000,1.000000
to,allenham,satisfied,1,0.000004,17.82407,0.200000,2.321928
she,started,as,1,0.000004,17.82407,0.500000,1.000000


In [344]:
M[2].loc[('captain','wentworth')].sort_values('n', ascending=False).head()

,n,p,i,cp,ci
w2,,,,,
s,26,0.000112,13.123630,0.132653,2.914270
<s>,24,0.000103,13.239107,0.122449,3.029747
was,15,0.000065,13.917179,0.076531,3.707819
in,8,0.000034,14.824070,0.040816,4.614710
had,7,0.000030,15.016715,0.035714,4.807355


In [345]:
M[2].loc[('anne','elliot')].sort_values('n', ascending=False).head()

,n,p,i,cp,ci
w2,,,,,
<s>,5,0.000022,15.502142,0.217391,2.201634
and,3,0.000013,16.239107,0.130435,2.938599
as,2,0.000009,16.824070,0.086957,3.523562
with,2,0.000009,16.824070,0.086957,3.523562
again,1,0.000004,17.824070,0.043478,4.523562


## Predict Sentences

### Get a list of test sentences

In [346]:
test_sentences = """
I love you
I love cars
I want to
Anne said to
said to her
he said to
she said to
said to him
she read the
she went to
robots fly ufos
""".split('\n')[1:-1]

### Convert list to TEST_SENT

In [347]:
TEST_SENT = pd.DataFrame({'sent_str':test_sentences})
TEST_SENT.index.name = 'sent_num'

In [348]:
TEST_SENT

,sent_str
sent_num,
0,I love you
1,I love cars
2,I want to
3,Anne said to
4,said to her
5,he said to
6,she said to
7,said to him
8,she read the


### Convert TEST_SENT to TEST_TOKEN

In [349]:
TEST_TOKEN = TEST_SENT.sent_str.str.split(expand=True).stack().to_frame('token_str')
TEST_TOKEN.index.names = ['sent_num', 'token_num']
TEST_TOKEN['term_str'] = TEST_TOKEN.token_str.str.replace(r'[\W_]+', '').str.lower()

In [350]:
TEST_TOKEN

token_str term_str
sent_num token_num                   
0        0                 I        i
         1              love     love
         2               you      you
1        0                 I        i
         1              love     love
         2              cars     cars
2        0                 I        i
         1              want     want
         2                to       to
3        0              Anne     anne
         1              said     said
         2                to       to
4        0              said     said
         1                to       to
         2               her      her
5        0                he       he
         1              said     said
         2                to       to
6        0               she      she
         1              said     said
         2                to       to
7        0              said     said
         1                to       to
         2               him      him
8        0               she      she
         1              read     read
         2               the      the
9        0               she      she
         1              went     went
         2                to       to
10       0            robots   robots
         1               fly      fly
         2              ufos     ufos

### Extract TEST_NGRAMS from TEST_TOKEN

In [351]:
# TEST_TOKEN

In [352]:
TEST_NGRAMS = get_ngrams(TEST_TOKEN, n=3, sent_key='sent_num')

In [353]:
TEST_NGRAMS.head(10)

w0    w1    w2
sent_num token_num                  
0        0           <s>     i  love
         1             i  love   you
         2          love   you   <s>
         3           you   <s>   <s>
         4           <s>   <s>     i
1        0           <s>     i  love
         1             i  love  cars
         2          love  cars   <s>
         3          cars   <s>   <s>
         4           <s>   <s>     i

### Test the model

We test by joining the test ngrams with the model and then saving aggregate statistics to the sentence dataframe.

Note that testing is a special case of the split-apply-combine pattern.

In [354]:
def test_model(model, test_ngrams):

    # Get the model level and info feature
    n = len(model.index.names) - 1 
    f = 'c' * bool(n) + 'i'        

    # Do the test by join and then split-apply-combine
    T = test_ngrams.join(model[f], on=model.index.names).fillna(model[f].max()).copy()
    
    R = T.groupby('sent_num')[f].agg(['sum','mean'])
    R['pp'] = np.exp2(R['mean'])
    
    return R

### Run tests and save as RESULT

In [355]:
# TEST_NGRAMS

In [356]:
RESULT = pd.concat(
    [test_model(M[i], TEST_NGRAMS.iloc[:,:i+1]) for i in range(len(M))],
    keys=[f"M{n}" for n in range(len(M))],
    axis=1
)

In [357]:
RESULT.style.background_gradient()

Show results for a given model.

In [358]:
pd.concat([TEST_SENT, RESULT['M2']], axis=1).sort_values('pp').style.background_gradient()

,sent_str,sum,mean,pp
sent_num,,,,
7,said to him,22.947928,4.589586,24.077033
4,said to her,25.400683,5.080137,33.827779
0,I love you,27.052688,5.410538,42.533792
5,he said to,28.679129,5.735826,53.291214
6,she said to,29.812309,5.962462,62.356228
9,she went to,38.371822,7.674364,204.274383
3,Anne said to,40.232394,8.046479,264.381761
2,I want to,42.494785,8.498957,361.777006
1,I love cars,52.935451,10.587090,1538.266952


Compare a feature across models.

We use `.swaplevel()` to change the order of the column levels to make selection easy.

In [359]:
RESULT.swaplevel(axis=1).style.background_gradient()

,sum,mean,pp,sum,mean,pp,sum,mean,pp
,M0,M0,M0,M1,M1,M1,M2,M2,M2
sent_num,,,,,,,,,
0,30.317702,6.063540,66.881740,23.072274,4.614455,24.495670,27.052688,5.410538,42.533792
1,41.146632,8.229326,300.105601,43.785131,8.757026,432.640857,52.935451,10.587090,1538.266952
2,29.074815,5.814963,56.296097,26.528941,5.305788,39.555000,42.494785,8.498957,361.777006
3,28.766060,5.753212,53.937325,29.798316,5.959663,62.235389,40.232394,8.046479,264.381761
4,25.868955,5.173791,36.096597,21.058451,4.211690,18.528706,25.400683,5.080137,33.827779
5,26.720884,5.344177,40.621648,25.733349,5.146670,35.424359,28.679129,5.735826,53.291214
6,26.310545,5.262109,38.375378,26.033164,5.206633,36.927732,29.812309,5.962462,62.356228
7,27.625431,5.525086,46.048628,21.039569,4.207914,18.480267,22.947928,4.589586,24.077033


In [360]:
RESULT.swaplevel(axis=1)['pp'].style.background_gradient()

,M0,M1,M2
sent_num,,,
0,66.881740,24.495670,42.533792
1,300.105601,432.640857,1538.266952
2,56.296097,39.555000,361.777006
3,53.937325,62.235389,264.381761
4,36.096597,18.528706,33.827779
5,40.621648,35.424359,53.291214
6,38.375378,36.927732,62.356228
7,46.048628,18.480267,24.077033
8,66.102319,501.219175,1767.004717


In [361]:
RESULT.swaplevel(axis=1)['mean'].style.background_gradient()

,M0,M1,M2
sent_num,,,
0,6.063540,4.614455,5.410538
1,8.229326,8.757026,10.587090
2,5.814963,5.305788,8.498957
3,5.753212,5.959663,8.046479
4,5.173791,4.211690,5.080137
5,5.344177,5.146670,5.735826
6,5.262109,5.206633,5.962462
7,5.525086,4.207914,4.589586
8,6.046629,8.969298,10.787090


### Compute Model Perplexity

In [362]:
np.exp2(RESULT.swaplevel(axis=1)['mean'].mean())

M0     86.471307
M1     92.523517
M2    227.205460
dtype: float64

In language modeling, perplexity is a measure of how well a probability model predicts a test set. It is often used to compare different models: the lower the perplexity, the better the model's performance in terms of predicting the test set. When dealing with n-gram models like unigrams, bigrams, and trigrams, the relationship between the value of 'n' and the perplexity of the model on a given test corpus can vary, depending on several factors.

1. **Data Sparsity**: As 'n' increases in n-gram models (moving from unigram to bigram to trigram), the models become more sensitive to the specific sequences of words in the training data. This can lead to an issue known as data sparsity: trigram models, for example, require a lot more data to encounter all possible word sequences of length three. If your training corpus isn't sufficiently large and diverse, trigram and higher n-gram models may suffer because they haven't seen enough examples of each word sequence during training. 

2. **Generalization vs. Specificity**: Unigram models, being the simplest, have high generalizability but low specificity—they treat each word independently and don't capture the context. Trigram models, on the other hand, capture more context but might become too specific to the training data, failing to generalize well to unseen data. If the test corpus contains many word sequences not seen in the training corpus, a higher n-gram model might perform poorly, leading to higher perplexity.

3. **Model Complexity and Overfitting**: Higher n-gram models (like trigrams) are more complex and can potentially overfit the training data. Overfitting occurs when a model learns patterns that are specific to the training data, including noise and outliers, rather than capturing the underlying structure of the language. This can lead to increased perplexity on the test set, as the model is less able to generalize to unseen data.

In summary, whether perplexity increases or decreases with n depends on the characteristics of your training and test data, as well as how well the model's complexity is suited to the amount and diversity of the data. In an ideal scenario with ample, diverse training data, you might expect a bigram or trigram model to outperform a unigram model, leading to lower perplexity. However, in practical scenarios, especially when dealing with limited or highly specific datasets, the relationship might not be so straightforward.

## Bonus: Generate Text

We use so-called "stupid back-off" to account for missing ngrams.

In [363]:
def generate_text(n=250):
    
    m1, m2, m3 = M
    start_word = m1.sample(weights='p').index[0][0]
    words = [start_word]
    
    for i in range(n):
        
        if len(words) == 1:
            next_word = m2.loc[start_word].sample(weights='p').index[0]
        
        elif len(words) > 1:

            # Get previous two words
            bg = tuple(words[-2:])
            
            # Try trigram model
            try:
                next_word = m3.loc[bg].sample(weights='cp').index[0]
            
            # If not in model, back off ...
            except KeyError:
                
                # Get the last word in the bigram
                ug = bg[1] 

                if ug == '<s>':
                    next_word = m1.sample(weights='p').index[0][0]
                else:
                    next_word = m2.loc[ug].sample(weights='cp').index[0]
            
        # Some words are returned as single item tuples
        if isinstance(next_word, tuple):
            next_word = next_word[0]
        
        words.append(next_word)
    
    text = ' '.join(words)
    lines = text.split('<s> <s>')
    for line in lines:
        print(line.strip().upper())

In [364]:
# M[1].loc['<s>']

In [365]:
generate_text()

AND EVEN WHEN HER SPIRITS WERE QUITE UNKNOWN TO MRS
ELINOR FOR WILLOUGHBY TO BE SPEEDY
BUT IT WAS A STRUGGLE
AND HOW DOES HE TRAVEL
SAID SHE AFTER THE LITTLE BOY
SHE THANKED HEAVEN THAT SHE IS A RECOLLECTION WHICH OUGHT TO GO IMMEDIATELY TO THEIR BLESSINGS IF SHE WERE TO BE EXPRESSED
THEY WERE BROUGHT IN AND PUT IN HIS BROTHERS AND SISTERS IN LAW WERE DEGRADED TO THE COTTAGE TO BE EMPLOYED WAS EASILY WHILED AWAY IN ABOUT FIVE MINUTES BROUGHT A GLASS OF WINE WHICH ELINOR PROCURED FOR HER BIRTH
HERE WERE FUNDS OF ENJOYMENT
AND SOON BE ON CORDIAL TERMS WITH THE VIEW OF THE FAMILY IT WAS A SINGLE MAN I LIKE
HIS IMPRUDENCE HAD MADE A POINT OF CONCLUDING IT WHEN YOU ALMOST BROKE IT TO HER
BUT MRS
HARRIS AT FOUR O CLOCK THIS MORNING AND MUST ABIDE THE CONSEQUENCES OF HER ELDEST DAUGHTER WHOSE STEADIER JUDGMENT REJECTED SEVERAL HOUSES AS TOO LITTLE
WHILE ADMIRAL CROFT S GIG
SHE TURNED THROUGH THE SUBJECT WITH AN ELLIOT AND SIR WALTER ELLIOT S MANNERS BETTER THAN HER YOUNG FRIEND S USUAL STATE 

In [370]:
generate_text()

SENTENCES BEGUN WHICH HE COULD REFER SIR WALTER ELLIOT HAS EYES UPON HIM WHICH GENERAL ATTENTION AND SHE MIGHT THINK ONLY OF AVOIDING CAPTAIN WENTWORTH HE NEED NOT BE SO UNJUST NOR SO INACCESSIBLE TO ALL HER GOOD NATURE WOULD GO
IF WE CAN ASK
THE ARRIVAL OF THE WORLD WHOM I LOVED HIM
WHICH AFTERWARDS WHEN HIS ASSURANCES HIS FELICITATIONS ON A DECENT EDUCATION WAS BROUGHT TO ME
BUT MRS
DID I EVER KNOWN IT MANY WEEKS WE HAD BETTER LEAVE THE ROOM VACANT WE MIGHT NOT BE VERY MUCH FORGOTTEN
KNOW HIM THEN AS I OUGHT TO BE CONSIDERED AS THE IDEAS OF DANGER
FOR WHATEVER OTHER CLAIMS MIGHT BE THE FARTHER KNOWLEDGE OF THE HARP WHICH WAS SEEN AGAIN IN NERVOUS GRATITUDE
THE INTERRUPTION TO PROCEED VERY SOON HAVE A VIEW OF IT ALL REPEATING HER CONVICTION OF THIS
IT WOULD BE GLAD TO SPARE A HUNDRED A YEAR FOR ANY PLACE COULD GIVE HER EASE
NOW THERE IS NO USE ON THESE OCCASIONS AS SHE HAD BEEN ZEALOUSLY DISCHARGING ALL THE DELIGHT OF LUCY IN WHICH SHE WAS ENTREATED TO BE AT BATH ON WEDNESDAY AND WE S

## Save

In [367]:
path_prefix = f"{output_dir}/austen-combo"
NG.to_csv(f"{path_prefix}-NG.csv")
for i in range(len(M)):
    M[i].to_csv(f"{path_prefix}-M{i}.csv", index=True)